In [11]:
import pathlib

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [13]:
from orchid import (project_loader as opl)

In [14]:
import toolz.curried as toolz

In [15]:
import UnitsNet

In [16]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [17]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

{'permian-a': WindowsPath('c:/src/Orchid.IntegrationTestData/Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac'),
 'gng': WindowsPath('c:/src/Orchid.IntegrationTestData/GnG_DemoProject_wDataFrames.ifrac')}

In [18]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

{'permian-a': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x00000154C57DE1F0>,
 'gng': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x00000154C57DE1C0>}

In [19]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

{'permian-a': <Orchid.FractureDiagnostics.UnitSystem object at 0x00000154C785CC10>,
 'gng': <Orchid.FractureDiagnostics.UnitSystem object at 0x00000154C785CCD0>}

In [33]:
def accumulator(so_far, element):
    return toolz.assoc(so_far,  element.ObjectId, element)

def data_frame_map(project):
    return toolz.pipe(project.DataFrames.Items,
                   list,
                   lambda dfs: toolz.reduce(accumulator, dfs, {})
              )

net_data_frames = toolz.valmap(data_frame_map, projects)
net_data_frames

{<System.Guid object at 0x00000154E0B138B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E0B13940>}
{<System.Guid object at 0x00000154E0B138B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E0B13940>, <System.Guid object at 0x00000154E0B282B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x00000154E0B139A0>}
{<System.Guid object at 0x00000154E0B138B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E0B13940>, <System.Guid object at 0x00000154E0B282B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x00000154E0B139A0>, <System.Guid object at 0x00000154E0B28C10>: <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x00000154E0B13BB0>}
{<System.Guid object at 0x00000154E0B288B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E

{'permian-a': {<System.Guid object at 0x00000154E0B138B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E0B13940>,
  <System.Guid object at 0x00000154E0B282B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x00000154E0B139A0>,
  <System.Guid object at 0x00000154E0B28C10>: <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x00000154E0B13BB0>},
 'gng': {<System.Guid object at 0x00000154E0B288B0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x00000154E0B28A00>,
  <System.Guid object at 0x00000154E0B28700>: <Orchid.FractureDiagnostics.Factories.DataFrames.FaultTraceSetDataFrame object at 0x00000154E0B28A30>,
  <System.Guid object at 0x00000154E0B28550>: <Orchid.FractureDiagnostics.Factories.DataFrames.StageDataFrame object at 0x00000154E0B289A0>,
  <System.Guid object at 0x00000154E0B28400>: <Orchid.FractureDiagnostics.Factories.DataFrames.WellLogSet

In [ ]:
def data_frame_identifying(data_frame):
    return data_frame.ObjectId, data_frame.Name, data_frame.DisplayName

identifications = toolz.valmap(toolz.valmap(data_frame_identifying), net_data_frames)
identifications